Requirements
* time_series_covid19_confirmed_US.csv
* time_series_covid19_deaths_US.csv
* governers usa.csv

In [2]:
#Libraries
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Read Data
df_conf=pd.read_csv("time_series_covid19_confirmed_US.csv")
#df_death=pd.read_csv("time_series_covid19_deaths_US.csv")
df_gov=pd.read_csv("governers usa.csv", encoding='iso-8859-1')

def flourish_preprocessor(df_conf):
    #Select US
    fl_conf=df_conf[df_conf.iso2=='US']

    #Drop unwanted columns
    fl_conf.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2','Country_Region', 'Lat', 'Long_','Combined_Key'], axis=1, inplace=True)

    #Make dataframe
    keys=fl_conf.columns
    cols=list([keys[0]])+list(['Party'])+list(keys[1:])
    df_fl=pd.DataFrame(columns=cols)
    #Province
    df_fl.Province_State=fl_conf.Province_State.unique()
    #Values
    for state in df_fl.Province_State:
        for col in keys[1:]:
            df_fl[col][df_fl.Province_State==state]=fl_conf[col][fl_conf.Province_State==state].sum()
    #Party        
    for state in df_fl.Province_State:
        df_fl.Party[df_fl.Province_State==state]=df_gov.Party[df_gov.State==state].sum()
    df_fl.Party[df_fl.Province_State=='District of Columbia']="Democratic"
    #Drop Non-States(Cruise Ships)
    df_fl=df_fl.drop(df_fl[df_fl.Party==0].index)

    return df_fl

fl_conf=flourish_preprocessor(df_conf)
#fl_death=flourish_preprocessor(df_death)
fl_conf.to_csv('fl_US_Conf_StateWise.csv', index=None)
#fl_death.to_csv('fl_US_Death_StateWise.csv', index=None)